In [18]:

from pathlib import Path
from benchmark import BenchmarkCase

bm = 'autochip'

if bm == 'verilog-eval':
    bmcase = BenchmarkCase(
        'Prob030_popcount255', 
        'benchmarks/VerilogEval_sample/Prob030_popcount255_spec.txt',
        'benchmarks/VerilogEval_sample/Prob030_popcount255_ref.sv',
        'benchmarks/VerilogEval_sample/Prob030_popcount255_tb.sv'
    )
    output_dir = Path('output/VerilogEval/')
    output_dir.mkdir(parents=True, exist_ok=True)

elif bm == 'autochip':
    bmcase = BenchmarkCase(
        'Vector5',
        'benchmarks/AutoChip_sample/Vector5_spec.txt',
        None, # No reference file provided
        'benchmarks/AutoChip_sample/Vector5_0_tb.v'
    )
    output_dir = Path('output/AutoChip/')
    output_dir.mkdir(parents=True, exist_ok=True)


print(bmcase.specification)

I would like you to implement a module named `TopModule` with the following interface. 
All input and output ports are one bit wide unless otherwise specified.

Module Name: `TopModule`
Ports:
  input a,
  input b,
  input c,
  input d,
  input e,
  output [24:0] out

Given five 1-bit signals (a, b, c, d, and e), compute all 25 pairwise one-bit comparisons in the 25-bit output vector. 
The output should be 1 if the two bits being compared are equal.

Hint: 
out[24] = ~a ^ a;   // a == a, so out[24] is always 1.
out[23] = ~a ^ b;
out[22] = ~a ^ c;
...


In [19]:

from rechisel import initial_chisel_generation
from rechisel_chisel import ChiselCode


target_path = output_dir / 'gen.scala'

if target_path.exists():
    chisel_code = ChiselCode(f"```scala\n\n{target_path.read_text()}\n\n```", 'TopModule')
else:
    chisel_code = initial_chisel_generation(bmcase, 'TopModule')
    target_path.write_text(chisel_code.decorated)

print(chisel_code.decorated)


package TopModule

import chisel3._
import chisel3.util._

import chisel3.stage.ChiselStage

class TopModule extends RawModule {
  // Input ports
  val a = IO(Input(Bool()))
  val b = IO(Input(Bool()))
  val c = IO(Input(Bool()))
  val d = IO(Input(Bool()))
  val e = IO(Input(Bool()))

  // Output port
  val out = IO(Output(UInt(25.W)))

  // Compute pairwise comparisons
  out := Cat(
    ~a ^ e,  // out[0] = e == a
    ~a ^ d,  // out[1] = d == a
    ~a ^ c,  // out[2] = c == a
    ~a ^ b,  // out[3] = b == a
    ~a ^ a,  // out[4] = a == a (always 1)
    ~b ^ e,  // out[5] = e == b
    ~b ^ d,  // out[6] = d == b
    ~b ^ c,  // out[7] = c == b
    ~b ^ a,  // out[8] = a == b
    ~b ^ b,  // out[9] = b == b (always 1)
    ~c ^ e,  // out[10] = e == c
    ~c ^ d,  // out[11] = d == c
    ~c ^ b,  // out[12] = b == c
    ~c ^ a,  // out[13] = a == c
    ~c ^ c,  // out[14] = c == c (always 1)
    ~d ^ e,  // out[15] = e == d
    ~d ^ c,  // out[16] = c == d
    ~d ^ b,  // out[17] = b 

In [20]:

from rechisel_verifier import Verifier, VerifierWorkingSpace, VerifyResult


verifier = Verifier(
    VerifierWorkingSpace(str(output_dir / 'chisel'), str(output_dir / 'iv')),
    bm
)

is_ok = (
    verifier.prepare(chisel_code, bmcase) and
    verifier.compile() and
    verifier.iv() and
    verifier.vvp() and
    verifier.functionality()
)
verify_result: VerifyResult = verifier.result

is_ok

False

In [21]:
import json

with open(output_dir / 'verify.json', 'w') as fp:
    json.dump(verify_result.__dict__, fp, indent=2)